In [ ]:
#Raneem Ali

In [ ]:
# Run cell to import libraries 
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt
import contextily 
import mapclassify 
import folium
import aiohttp
import fsspec

# This loads geodataframe contaning county geometry shapes
c = "https://github.com/babdelfa/gis/blob/main/counties_geometry.zip?raw=true"
import fsspec
with fsspec.open(c) as file:
    county_shapes = gpd.read_file(file)

# This loads the most recent covid19 data from Johns Hopkins University's Github
url_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
df_cases = pd.read_csv(url_cases) 
url_deaths = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df_deaths = pd.read_csv(url_deaths) 



In [ ]:

# Drop/Rename  
drop = ["UID", "iso2", "iso3", "code3", "Country_Region","Lat", "Long_"]

df_cases.drop(columns=drop, inplace=True)
df_deaths.drop(columns=drop, inplace=True)
df_deaths.drop(columns=['Population'], inplace=True)

rename = {"Admin2": "County", "Province_State" : "state", "Combined_Key" : "county_state", "deaths":"Total Deaths", "cases":"Total Cases"}

df_cases.rename(columns=rename, inplace=True)
df_deaths.rename(columns=rename, inplace=True)
df_cases
# Melt
keep = ["County", "state", "county_state","FIPS"]
df_cases_melted = pd.melt(df_cases, id_vars=keep, 
                          var_name='date', 
                          value_name='Total Cases')
df_cases_melted.tail()

df_deaths_melted = pd.melt(df_deaths, id_vars=keep, 
                           var_name='date', 
                           value_name='Total Deaths')
df_deaths_melted.tail()

# Merge
df = pd.merge(df_cases_melted, df_deaths_melted)
df.tail()

# DateTime obj
df_cases_melted.date = pd.to_datetime(df_cases_melted['date'])
df.date = pd.to_datetime(df_cases_melted['date'])

# Group by

usa_cases = df.groupby(df['date'])["Total Cases"].sum()
usa_deaths = df.groupby(df['date'])["Total Deaths"].sum()


In [ ]:
# Project code & output

# Enter name
name = input("Enter your name: ")
print("Hello "+name)

# User info
print("\nInteractive Covid-19 Map of the US")
print("By: "+name)
from datetime import date
print("Date:", date.today().strftime("%B %d, %Y"))

# General info
import datetime
date= pd.to_datetime(df['date']).max().strftime("%m/%d/%y")
print("\nGeneral data information:")
print("\tTopic: COVID-19 reported cases & death")
print("\tData source: Johns Hopkins University")
print("\tTime period: "+date)

# USA overview
trc = usa_cases.max()
trd = usa_deaths.max()
print("\nUSA OVERVIEW")
print("\t- Total reported cases: " + f"{trc:,}")
print("\t- Total reported deaths: " + f"{trd:,}")

# State input
print("\n\n"+name+", enter a state name for more COVID-19 information:")
user_input = input("Enter state: ").capitalize()
user_state = df[df.state == user_input].copy()
print("Found information for "+ user_input+".")


# State overview
state_trc = user_state.groupby(['date'])["Total Cases"].sum().max()
state_trd = user_state.groupby(['date'])["Total Deaths"].sum().max()
print("\n\n"+user_input.upper()+" OVERVIEW")
print("\t- Total reported cases: " + f"{state_trc:,}")
print("\t- Total reported deaths: " + f"{state_trd:,}")

# Interactive map
print("\n\nBelow is an interactive map of COVID-19 cases and deaths for "+user_input+" counties.")

county_shapes = county_shapes[['FIPS_BEA', 'geometry']].copy()

####filter date on df here#####
dd = df[(df["date"]==pd.to_datetime(df['date']).max()) & (df.state == user_input)].copy()
#### code below can stay as is
gdf = pd.merge(county_shapes, dd, left_on="FIPS_BEA", right_on="FIPS")
gdfs = gdf[["County", "Total Cases", "geometry","Total Deaths"]].copy()
gdfs.explore('Total Deaths', cmap="Set2", legend=True, scheme="NaturalBreaks") # <----uncomment addressing comments below
